In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("discussion.ipynb")

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Discussion 9

### Due Saturday November 20th, 11:59:59PM


# Scikit-Learn: Transformers, Estimators, and Pipelines

In [3]:
# import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from discussion import *

# Transformers and Estimators

Scikit learn includes two *base* modeling classes: Transfomers and Estimators. Both are classes that are meant to be *fit* on (training) data and then later used to transform (or predict with) unseen data.

### Transformers

* Transformers take input data and transform it into output data via the `transform` method.
* Sometimes transformers need prior information (parameters) about the data before transforming it.
    - In this case, the transformer is *fit* using the `fit` method on training data to estimate the parameters.
    - Once fit, the transformer may then be applied to `test` data (or unseen, new data).
* Fit parameters are accessed via an instance variable that ends in an *underscore*.

**Question 1** Using `Binarizer`, transform the `city-mpg` and `highway-mpg` column to 0 if the mpg is less than or equal to 25 and 1 if it's greater than 25.

In [18]:
cars = pd.read_csv('data/cars.csv')
cars.head()

,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
from sklearn.preprocessing import Binarizer

In [8]:
binarizer = Binarizer(threshold = 25)
highway_mgp_bin = binarizer.transform(cars[['highway-mpg']])
highway_mgp_bin        

array([[1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

**Question 2** Using `FunctionTransformer`, transform the `city-mpg` and `highway-mpg` columns to a log-scale.

In [9]:
from sklearn.preprocessing import FunctionTransformer

In [11]:
func = FunctionTransformer(np.log)
func.transform(cars['city-mpg'])

0      3.044522
1      3.044522
2      2.944439
3      3.178054
4      2.890372
         ...   
188    3.135494
189    2.944439
190    2.890372
191    3.258097
192    2.944439
Name: city-mpg, Length: 193, dtype: float64

In [ ]:
def my_log(x):
    return np.log(x) + 10

Most transformers you will use will require being *fit to training data* before using it. An example of this is one-hot-encoding: before applying one-hot encoding to a column, you must determine the number of distinct values in the column (as that number determines the number of columns in the output).

**Question 3** *(Fit transformers properly handle unseen values)*

1. One-hot encode the `body-style` column using `OneHotEncoder`. What is the dimension of the output? Which column corresponds to which value of `body-style`? (If you can't remember the attribute name, look up the documentation!)
1. Fit a `OneHotEncoder` on the *first 5 rows* of the `body-style` column. Use this 'training data' to one-hot-encode the `body-style` in rest of the dataset. Why does it throw an exception? Look at the documentation -- what is the relevant parameter to avoid this? What are the implications of setting this parameter? What is the dimension of the output?

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
cars['body-style'].unique()

array(['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
      dtype=object)

In [14]:
enc = OneHotEncoder()
enc.fit(cars[['body-style']].iloc[:5,])

OneHotEncoder()

In [15]:
enc.categories

'auto'

In [23]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(cars[['body-style']].iloc[:5,])

OneHotEncoder(handle_unknown='ignore')

In [24]:
enc.fit(cars[['body-style']].iloc[:5,])

OneHotEncoder(handle_unknown='ignore')

In [26]:
enc.transform(cars[['body-style']]).toarray()

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1

As you observed, the categories of `OneHotEncoder` are learned from training data and saved as an attribute of the transformer. These categories are then used to transform new, unseen data which is then used by other pieces of the ML pipeline.

Below is an illustration of why *fitting* a transformer is so important:

*The dangers of `pd.get_dummies`*: Pandas offers it's own one-hot encoder called `get_dummies`. This function is stateless; every time it's called, it determines the categories of the input data and one-hot encodes that data using those categories. However, as you saw in the above question, this is not a realistic use of the function!

To illustrate this:
1. We will create a one-hot encoding using scikit-learn that we will pass into a linear regression model.
1. We will create a *stateless* one-hot encoder that we will pass into a linear regression model.

Both of these models will be trained on the first 5 rows of the dataset; the rest will be used as 'unseen' data.

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
X_train, y_train = cars[['body-style']].head(5), cars['price'].head(5)
X_test, y_test = cars[['body-style']].tail(-5), cars['price'].tail(-5)

In [10]:
# Using sklearn transformers. What are the categories of the OneHotEncoder?
ohe = OneHotEncoder(handle_unknown='ignore')
lr = LinearRegression()

ohe.fit(X_train)
features = ohe.transform(X_train)
lr.fit(features, y_train)

In [11]:
# predict on the new data using the fit model:
lr.predict(ohe.transform(X_test))[:10]

In [12]:
# using a stateless one-hot encoder.
ohe = FunctionTransformer(pd.get_dummies, validate=False)
lr = LinearRegression()

ohe.fit(X_train)
features = ohe.transform(X_train)
lr.fit(features, y_train)

In [13]:
# debug this!
# lr.predict(ohe.transform(X_test))[:10]

**Note:** Even worse, there are cases where such an ML pipeline doesn't even throw an exception -- the incorrect columns get silently passed on. 

**The below predictions are wrong! Can you tell that it's wrong?** Debugging statistical output is notoriously hard -- this is why statistical analysis of the output data is always an important step to check your work!

In [14]:
X_train, y_train = cars[['body-style']].head(5), cars['price'].head(5)
X_test, y_test = cars[['body-style']].iloc[5:20], cars['price'].iloc[5:20]

In [15]:
ohe = FunctionTransformer(pd.get_dummies, validate=False)
lr = LinearRegression()

ohe.fit(X_train)
features = ohe.transform(X_train)
lr.fit(features, y_train)

In [16]:
# THIS IS WRONG! debug this! 
lr.predict(ohe.transform(X_test))[:10]

# Building custom Transformers

Building your own transformer class is easy! There is are convenient base-classes you can inherit: `TranformerMixin` and `BaseEstimator`. To subclass these classes, you need to:
1. implement the `fit` method, and
2. implement the `transform` method.
Once you have done that, you can use your custom transformer as part of a `Pipeline` that can leverage all the nice features of scikit-learn (like feature-selection libraries and cross-validation).

To get acquainted with the structure of a transformer, it's useful to look at `sklearn` source code. First, you will walk through the source code of the `Binarizer` transformer ([source code](https://github.com/scikit-learn/scikit-learn/blob/7813f7efb/sklearn/preprocessing/data.py#L1789)).

The source code is included below. Note, that there is a lot of boiler-plate code, but the *transform* method is the relevant method. (Why does the fit method do nothing?)

```
class Binarizer(BaseEstimator, TransformerMixin):
    """Binarize data (set feature values to 0 or 1) according to a threshold
    Values greater than the threshold map to 1, while values less than
    or equal to the threshold map to 0. With the default threshold of 0,
    only positive values map to 1.
    Binarization is a common operation on text count data where the
    analyst can decide to only consider the presence or absence of a
    feature rather than a quantified number of occurrences for instance.
    It can also be used as a pre-processing step for estimators that
    consider boolean random variables (e.g. modelled using the Bernoulli
    distribution in a Bayesian setting).
    Read more in the :ref:`User Guide <preprocessing_binarization>`.
    Parameters
    ----------
    threshold : float, optional (0.0 by default)
        Feature values below or equal to this are replaced by 0, above it by 1.
        Threshold may not be less than 0 for operations on sparse matrices.
    copy : boolean, optional, default True
        set to False to perform inplace binarization and avoid a copy (if
        the input is already a numpy array or a scipy.sparse CSR matrix).
    Examples
    --------
    >>> from sklearn.preprocessing import Binarizer
    >>> X = [[ 1., -1.,  2.],
    ...      [ 2.,  0.,  0.],
    ...      [ 0.,  1., -1.]]
    >>> transformer = Binarizer().fit(X)  # fit does nothing.
    >>> transformer
    Binarizer(copy=True, threshold=0.0)
    >>> transformer.transform(X)
    array([[1., 0., 1.],
           [1., 0., 0.],
           [0., 1., 0.]])
    Notes
    -----
    If the input is a sparse matrix, only the non-zero values are subject
    to update by the Binarizer class.
    This estimator is stateless (besides constructor parameters), the
    fit method does nothing but is useful when used in a pipeline.
    See also
    --------
    binarize: Equivalent function without the estimator API.
    """

    def __init__(self, threshold=0.0, copy=True):
        self.threshold = threshold
        self.copy = copy

    def fit(self, X, y=None):
        """Do nothing and return the estimator unchanged
        This method is just there to implement the usual API and hence
        work in pipelines.
        Parameters
        ----------
        X : array-like
        """
        check_array(X, accept_sparse='csr')
        return self

    def transform(self, X, copy=None):
        """Binarize each element of X
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape [n_samples, n_features]
            The data to binarize, element by element.
            scipy.sparse matrices should be in CSR format to avoid an
            un-necessary copy.
        copy : bool
            Copy the input X or not.
        """
        copy = copy if copy is not None else self.copy
        return binarize(X, threshold=self.threshold, copy=copy)

    def _more_tags(self):
        return {'stateless': True}
```

The relevant (portion of the) function `binarize` from the transform method is here:

```
def binarize(X, threshold=0.0, copy=True):
    ...
    cond = X > threshold
    not_cond = np.logical_not(cond)
    X[cond] = 1
    X[not_cond] = 0
    return X
```

**Question 4** 

As a warm-up, create a transformer that drops the `i`th row of an input dataset.

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnDropper(BaseEstimator, TransformerMixin):

    def __init__(self, index):
        self.index = index

    def fit(self, X, y=None):
        """
        Does nothing! Stateless!
        """
        return self

    def transform(self, X, y=None):
        """
        Drops the ith column of X, where i=index
        """
        
        return np.delete(np.array(X), self.index, 1)

In [28]:
cd = ColumnDropper(index=3)
cd.transform(cars.iloc[:,:5])

array([['alfa-romero', 'gas', 'std', 'convertible'],
       ['alfa-romero', 'gas', 'std', 'convertible'],
       ['alfa-romero', 'gas', 'std', 'hatchback'],
       ['audi', 'gas', 'std', 'sedan'],
       ['audi', 'gas', 'std', 'sedan'],
       ['audi', 'gas', 'std', 'sedan'],
       ['audi', 'gas', 'std', 'sedan'],
       ['audi', 'gas', 'std', 'wagon'],
       ['audi', 'gas', 'turbo', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['bmw', 'gas', 'std', 'sedan'],
       ['chevrolet', 'gas', 'std', 'hatchback'],
       ['chevrolet', 'gas', 'std', 'hatchback'],
       ['chevrolet', 'gas', 'std', 'sedan'],
       ['dodge', 'gas', 'std', 'hatchback'],
       ['dodge', 'gas', 'std', 'hatchback'],
       ['dodge', 'gas', 'turbo', 'hatchback'],
       ['

**Question 5**

Columns that don't have much variation are not very useful for prediction. An extreme case is when a column has only a single value.  Create a "feature selection" transformer that drops any columns that don't have a standard-deviation greater than an input threshold.

* What needs to be calculated during the fitting process? When is the standard deviation calculated?

In [ ]:
lvd = 

In [30]:
lvd = LowStdColumnDropper(thresh=10)
lvd.fit(cars.select_dtypes('number'))
out = lvd.transform(cars.select_dtypes('number'))

AttributeError: 'LowStdColumnDropper' object has no attribute 'transform'

In [40]:
out = LowStdColumnDropper(thresh=10)
out.fit_transform(cars.select_dtypes('number')).shape[0] == cars.shape[0]


True

In [39]:
out.fit_transform(cars).shape[0] == cars.shape[0]

IndexError: Boolean index has wrong length: 14 instead of 24

In [34]:
grader.check("q5")

q5 results:
    q5 - 1 result:
        Test case passed!

    q5 - 2 result:
        Trying:
            out.fit_transform(cars).shape[0] == cars.shape[0]
        Expecting:
            True
        **********************************************************************
        Line 1, in q5 1
        Failed example:
            out.fit_transform(cars).shape[0] == cars.shape[0]
        Exception raised:
            Traceback (most recent call last):
              File "E:\anaconda\lib\doctest.py", line 1336, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q5 1[0]>", line 1, in <module>
                out.fit_transform(cars).shape[0] == cars.shape[0]
              File "E:\anaconda\lib\site-packages\pandas\core\generic.py", line 5465, in __getattr__
                return object.__getattribute__(self, name)
            AttributeError: 'DataFrame' object has no attribute 'fit_transform'

    q5 - 2 message: doctest 1

    q5 - 3 result:
        Trying:
            out.fit_transform(cars).shape[1] <= cars.shape[1]
        Expecting:
            True
        **********************************************************************
        Line 1, in q5 2
        Failed example:
            out.fit_transform(cars).shape[1] <= cars.shape[1]
        Exception raised:
            Traceback (most recent call last):
              File "E:\anaconda\lib\doctest.py", line 1336, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q5 2[0]>", line 1, in <module>
                out.fit_transform(cars).shape[1] <= cars.shape[1]
              File "E:\anaconda\lib\site-packages\pandas\core\generic.py", line 5465, in __getattr__
                return object.__getattribute__(self, name)
            AttributeError: 'DataFrame' object has no attribute 'fit_transform'

    q5 - 3 message: doctest 2

    q5 - 4 result:
        Trying:
            out.fit_transform(cars).shape[1] == 14
        Expecting:
            True
        **********************************************************************
        Line 1, in q5 3
        Failed example:
            out.fit_transform(cars).shape[1] == 14
        Exception raised:
            Traceback (most recent call last):
              File "E:\anaconda\lib\doctest.py", line 1336, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q5 3[0]>", line 1, in <module>
                out.fit_transform(cars).shape[1] == 14
              File "E:\anaconda\lib\site-packages\pandas\core\generic.py", line 5465, in __getattr__
                return object.__getattribute__(self, name)
            AttributeError: 'DataFrame' object has no attribute 'fit_transform'

    q5 - 4 message: correct number of columns dropped for thresh=0.25

    q5 - 5 result:
        Trying:
            out1.fit_transform(cars).shape[1] == 12
        Expecting:
            True
        **********************************************************************
        Line 1, in q5 4
        Failed example:
            out1.fit_transform(cars).shape[1] == 12
        Exception raised:
            Traceback (most recent call last):
              File "E:\anaconda\lib\doctest.py", line 1336, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q5 4[0]>", line 1, in <module>
                out1.fit_transform(cars).shape[1] == 12
            NameError: name 'out1' is not defined

    q5 - 5 message: correct number of columns dropped for thresh=1.0

    q5 - 6 result:
        Trying:
            out2.fit_transform(cars).shape[1] == 6
        Expecting:
            True
        **********************************************************************
        Line 1, in q5 5
        Failed example:
            out2.fit_transform(cars).shape[1] == 6
        Exception raised:
            Traceback (most recent call last):
              File "E:\anaconda\lib

# Scikit-Learn Pipelines

Pipelines are ways of chaining transformers and estimators together. 
* A **Pipeline** object is a sequence of transformers that perhaps end with estimator.
* When you call `.fit(X, y)` on a pipeline, the pipeline calls `fit_transform` on each successive transformer in the pipeline, passing the transformed data to the next transformer in the sequence.
* A pipeline that consists of a sequence of transformers is itself a transformer.
* A pipeline that consists of a sequence of transformers, followed by an estimator, is itself an estimator.
    - These observations allow you put pipelines inside of other pipelines!

**Question 6** Create a *single* pipeline that consists of a `OneHotEncoder`, followed by a `LowStdColumnDropper`, followed by a `LinearRegression` model. That is, one-hot encode the categorical features, drop the low-variance columns, and use those features to fit a linear regression model. Your threshold for "small standard deviation" should be `0.1`. 

*Remark:* Be sure to pass `sparse=False` into `OneHotEncoder` (or else, make your `LowStdColumnDropper` handle sparse matrices).

How many columns did your pipeline drop? (Use the `named_steps` attribute!)



In [28]:
from sklearn.pipeline import Pipeline

### Putting columns together using `ColumnTransformer`

You can run many different transformers in parallel on different subsets of columns using `ColumnTransformer` (see the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer)).

One common pattern for using `ColumnTransformer` is to create a transformer pipeline for each kind of data in your dataset and use `ColumnTransformer` to put the transformed features together into a single dataset.

**Question 7**

Create a general pipeline that transforms different data-kinds with appropriate generic features (e.g. one-hot-encoder, ordinal-encoder) by filling in the `...` below:

In [29]:
from sklearn.compose import ColumnTransformer

In [30]:
quantitative_cols = ...
ordinal_cols = ...
nominal_cols = ...

In [31]:
# quantitative_pipeline = Pipeline([...])
# ordinal_pipeline = Pipeline([...])
# nominal_pipeline = Pipeline([...])

In [32]:
# feature_eng_pipeline = ColumnTransformer([
#     ('quant', quantitative_pipeline, quantitative_cols),
#     ('ordin', ordinal_pipeline, ordinal_cols),
#     ('nomin', nominal_pipeline, nominal_cols)
# ])

In [33]:
# pl = Pipeline([feature_eng_pipeline, LinearRegression()])

**Question 8** 

Fit the pipeline and predict with it. Check the outputs at each step using `named_steps`.

# Finished? Turn in Question 5 for Grading

## Congratulations! You're done!

* Submit your `.py` file to Gradescope. Note that you only need to submit the `.py` file; this notebook should not be uploaded. Make sure that all of your work is in the `.py` file and not here by running the doctests: `python -m doctest discussion.py`.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()